In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

In [ ]:
df = pd.read_csv('../input/planet-understanding-the-amazon-from-space/train_v2.csv')
labels = df['tags'].str.split(' ')

In [ ]:
from sklearn.preprocessing import MultiLabelBinarizer
mlb = MultiLabelBinarizer()
l = mlb.fit_transform(labels)

In [ ]:
# after getting the structure let's write a custom PyTorch Dataset 
# setitem and len are required by the parent class
import torch
from torch.utils.data import Dataset, DataLoader
import pandas as pd
from sklearn.preprocessing import MultiLabelBinarizer
from skimage import io
from torchvision import transforms

class AmazonDataset(Dataset):

    def __init__(self, file_name, filepath='../input/planet-understanding-the-amazon-from-space', transforms=None):
        """
        read the file and lazy load the images in getitem
        transforms: torchvision.transform.Compose
        """
        # headers = image_name, tags
        self.path = filepath
        self.datapath = os.path.join(self.path, '')
        self.image_df = pd.read_csv(os.path.join(self.path, file_name))
        self.X_train = self.image_df['image_name']
        # tags are space delimited
        self.mlb = MultiLabelBinarizer()
        self.y_train = self.mlb.fit_transform(self.image_df['tags'].str.split(' ')).astype('float')
        self.classes = list(self.mlb.classes_)
        self.transforms = transforms

    def __len__(self):
        """
        return the length of the training set
        """
        return len(self.image_df)

    def __getitem__(self, idx):
        """
        return the corresponding image and labels
        returns a tuple(image_tensor, label_tensor)
        """
        img_path = os.path.join(self.path, 'train-jpg', self.X_train.iloc[idx])
        image = io.imread(img_path + '.jpg')
        labels = torch.from_numpy(self.y_train[idx,:]).float()
        if transforms:
            image = self.transforms(image)
        return image, labels
    
    def label_dist(self, prop=False):
        tags = dataset.image_df['tags'].str.split(' ').to_list()
        tags = [items for tag in tags for items in tag]
        self.labels, self.counts = np.unique(tags, return_counts=True)


In [ ]:
transformations = transforms.Compose([
    transforms.ToPILImage(),
        transforms.Resize(32),
        transforms.RandomHorizontalFlip(),
        transforms.RandomRotation(20),
        transforms.ToTensor(),
#         transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])
dataset = AmazonDataset(file_name='train_v2.csv', transforms=transformations)

In [ ]:
train_loader = DataLoader(dataset, batch_size=64, shuffle=True)
use_cuda = torch.cuda.is_available()
device = torch.device("cuda:0" if use_cuda else "cpu")

In [ ]:
train_loader = DataLoader(dataset,
                          batch_size=256,
                          shuffle=True,
                          num_workers=4 # 1 for CUDA
                         # pin_memory=True # CUDA only
                         )

In [ ]:
from torchvision.models import vgg16
import torch.nn.functional as F
import torch.nn as nn

class Model(nn.Module):
    
    def __init__(self):
        super(Model, self).__init__()
        self.vgg = vgg16(pretrained=True)
        self.vgg = nn.Sequential(*list(self.vgg.children())[:-1])
        
        self.fc1 = nn.Linear(25088, 4096)
        self.fc2 = nn.Linear(4096, 1024)
        self.fc3 = nn.Linear(1024, 17)
        
    def forward(self,x):
        x = self.vgg(x)
        x = x.view(-1, np.prod(x.size()[1:]))
        x = F.dropout(F.relu(self.fc1(x)))
        x = F.dropout(F.relu(self.fc2(x)))
        x = self.fc3(x)
        return F.sigmoid(x)
        
        

In [ ]:
from torch import optim
model = Model()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [ ]:
model.to(device)

In [ ]:
def train(epoch, freeze=None, unfreeze=None):
    model.train()
    
    # 
    if freeze is not None:
        freeze = freeze*2
        for param in list(model.parameters())[:-freeze]:
            param.requires_grad = False
    if unfreeze:
        for param in list(model.parameters()):
            param.requires_grad = True
    
    for i in range(epoch):
        for a, (x, y) in enumerate(train_loader):
            x, y = x.to(device), y.to(device)
            optimizer.zero_grad()
            output = model(x)
            loss = F.binary_cross_entropy(output, y)
            loss.backward()
            optimizer.step()
            if a % 10 == 0:
                print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                    i, a * len(x), len(train_loader.dataset),
                    100. * a / len(train_loader), loss.item()))

In [ ]:
# check the accuracy on the train set
correct = 0
total = 0
with torch.no_grad():
    for i, data in enumerate(train_loader):
        if i!=1:
            images, labels = data
            images, lables = images.to(device), labels.to(device)
            outputs = model(images)
            print(outputs.data)
            _, predicted = torch.max(outputs.data, 1)
            print(predicted)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

print('Accuracy of the network on the 40479 train images: %d %%' % (100 * correct / total))